In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=fac3f977330b4c030a32bb23a83d89c74331203faf50fe80bd9b3f27d6b1427e
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 154.2 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My Drive/Colab Notebooks/NVIDIA-SuperSlowMo

/content/drive/My Drive/Colab Notebooks/NVIDIA-SuperSlowMo


In [3]:
!ls

 data   libs   Main.ipynb  'Train(mtahir).ipynb'   utils


In [0]:
import sys
sys.path.insert(0,'libs')
sys.path.insert(0,'utils')

In [5]:
import cv2
import gc
import os
import imageio
import math
import numpy as np
from Network import Network

Using TensorFlow backend.


In [0]:
class VideoGenerator():
    def __init__(self, vgg_file, weight_file, dims):
        self.shape = (dims[1], dims[0])
        self.Video_interpolator = Network(shape = dims, vgg_model = vgg_file, lr = 1e-4)
        self.Video_interpolator.load(weight_file)
    
    def slowMo_Video(self, in_video_file, out_video_file, fps_factor, seq_len):
        vid_obj = cv2.VideoCapture(in_video_file)
        fps = vid_obj.get(cv2.CAP_PROP_FPS)
        video = []
        t = [(1.0 / fps_factor) * i for i in range(1, fps_factor, 1)]
        T = [np.full((1,1,1,1), ti) for ti in t]
        
        if (seq_len == 0):
            seq_len  = math.inf
        frame = 0
        
        success , I0 = vid_obj.read()
        if not success:
                raise Exception('cannot read the video file')
            
        I0 = cv2.resize(I0, self.shape)
        video.append(I0)
        
        while (frame < seq_len):    
            success , I1 = vid_obj.read()
            if not success:
                break
                
            I1 = cv2.resize(I1, self.shape)            
            I0_in = np.expand_dims(self.normalize_in(I0), axis = 0)
            I1_in = np.expand_dims(self.normalize_in(I1), axis = 0)
            
            for i in range(len(T)):
                y = self.Video_interpolator.predict([T[i], I0_in, I1_in])[0]
                y = self.denormalize_out(y, to_rgb = False)
                video.append(y)
            
            video.append(I1)
            I0 = I1
            frame += 2
            gc.collect()
            print(frame)
        
        if (frame == 0 and not success):
            raise Exception('cannot read the video file')
            
        for i in range(len(video)):
          video[i] = cv2.cvtColor(video[i].astype(np.uint8), cv2.COLOR_BGR2RGB)
       
        print(len(video))
        video = np.asarray(video)
        
        imageio.mimwrite(out_video_file, video , fps = fps)
        
    def normalize_in(self, img, mean = [0, 0, 0], sd = [1, 1, 1], rescale = 1.0/255):
        out = img * rescale
        out = (out - mean) / sd
        return out
    
    def denormalize_out(self, img, mean = [0,0,0], sd = [1, 1, 1], rescale = 255, to_rgb = True):
        out = (img + mean) * sd
        if to_rgb:
          out = cv2.cvtColor(out.astype('float32'), cv2.COLOR_BGR2RGB)
        out *= rescale
        return out

In [7]:
weights = 'data/model_data/weights/'
Video_generator = VideoGenerator(weights + 'main/vgg16.h5', weights + 'mtahir/weights.46-26.9861.h5', dims =(384, 640, 3))

W0905 03:52:35.104467 140394117617536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0905 03:52:35.148628 140394117617536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0905 03:52:35.156661 140394117617536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0905 03:52:35.199108 140394117617536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0905 03:52:42.877330 140394117617536 deprecation_wrapp

In [8]:
filepath = 'data/'
filenames = [f for f in os.listdir(filepath)]
video_files = [f for f in filenames if any(filetype in f.lower() for filetype in ['.mp4', '.flv', '.webm', '.mov', '.m4v', '.mkv', '.gif'])]
vid = np.random.choice(video_files, 1, replace = True)[0]
vid_file = os.path.join(filepath, vid)
print(vid)

original.gif


In [9]:
fps = 8
Video_generator.slowMo_Video(vid_file, str(fps) + 'Xslow_'+vid, fps, 0) 

2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
241
